## <b><font color='darkblue'>Preface</font></b>
([source](https://cloud.google.com/architecture/choose-design-pattern-agentic-ai-system)) <font size='3ptx'><b>This document provides guidance to help you choose a design pattern for your agentic AI system.</b> Agent design patterns are common architectural approaches to build agentic applications. An agent design pattern offers a distinct framework for organizing a system's components, integrating the model, and orchestrating a single agent or multiple agents to accomplish a workflow.</font>

<font size='3ptx'><b>[AI agents](https://cloud.google.com/docs/generative-ai/glossary#ai-agents) are effective for applications that solve open-ended problems, which might require autonomous decision-making and complex multi-step workflow management</b></font>. Agents excel at solving problems in real-time by using external data and they excel at automating knowledge-intensive tasks. AI agents are suitable when you need AI to complete goal-focused tasks with some degree of autonomy. For other use cases, you can use assistive and generative AI applications. To learn about the differences between AI agents and non-agentic AI applications, see [What is the difference between AI agents, AI assistants, and bots?](https://cloud.google.com/discover/what-are-ai-agents#what-is-the-difference-between-ai-agents-ai-assistants-and-bots)

This guide assumes that you have a foundational knowledge of agentic AI systems and how their architecture differs from that of non-agentic systems, such as those that use direct model reasoning or [**retrieval-augmented generation** (RAG)](https://cloud.google.com/docs/generative-ai/glossary#retrieval-augmented-generation).

For a summary of the agent pattern guidance, see the [**compare design patterns section**](https://cloud.google.com/architecture/choose-design-pattern-agentic-ai-system#compare-design-patterns) later in this document.

### <b><font color='darkgreen'>Overview of the design process</font></b>
The following are the high-level steps to choose a design pattern for your agentic AI system. These steps are described in detail later in this document.
1. <b><font size='3ptx'>Define your requirements</font></b>: Assess the characteristics of your workload, including task complexity, latency and performance expectations, cost budget, and the need for human involvement.
2. <b><font size='3ptx'>Review the common agent design patterns</font></b>: Learn about the common design patterns in this guide, which include both single-agent systems and multi-agent systems.
3. <b><font size='3ptx'>Select a pattern</font></b>: Select the appropriate [**design pattern**](https://cloud.google.com/architecture/choose-design-pattern-agentic-ai-system#compare-design-patterns) based on your workload characteristics.

This process isn't a one-time decision. You should periodically revisit these steps to refine your architecture as your workload characteristics change, your requirements evolve, or new Google Cloud features become available.

### <b><font color='darkgreen'>Define your requirements</font></b>
<b><font size='3ptx'>The questions that follow aren't exhaustive checklists for planning</font></b>. Use these questions as a starting point to identify the primary goal of your agentic system and to select the best design pattern.
* <b><font size='3ptx'>Task characteristics</font></b>: Can your task be completed in predefined workflow steps or is the task open-ended? Does your task need to use an AI model to orchestrate the workflow?
* <b><font size='3ptx'>Latency and performance</font></b>: Do you need to prioritize fast or interactive responses at the cost of accuracy or high-quality responses? Or can your application tolerate a delay to achieve a more accurate or thorough result?
* <b><font size='3ptx'>Cost</font></b>: What is your budget for inference costs? Can you support patterns that require multiple calls to the model for a single request?
* <b><font size='3ptx'>Human involvement</font></b>: Does your task involve high-stakes decisions, safety-critical operations, or subjective approvals that require human judgment?

<b>If your workload is predictable or highly structured, or if it can be executed with a single call to an AI model, it can be more cost effective to explore non-agentic solutions for your task.</b> For example, you might not need an agentic workflow for tasks like summarizing a document, translating text, or classifying customer feedback. For information about choosing architecture components for generative AI applications that don't require an agentic infrastructure, see [**Choose models and infrastructure for your generative AI application**](https://cloud.google.com/docs/generative-ai/choose-models-infra-for-ai).

The following sections describe common agent design patterns for building a reliable and effective agentic AI system.

In [2]:
# --- Full example code demonstrating LlmAgent with Tools vs. Output Schema ---
import asyncio
import json # Needed for pretty printing dicts
      
from google.adk.agents import LlmAgent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types
from pydantic import BaseModel, Field
  
# --- 1. Define Constants ---
APP_NAME = "agent_comparison_app"
USER_ID = "test_user_456"
SESSION_ID_TOOL_AGENT = "session_tool_agent_xyz"
SESSION_ID_SCHEMA_AGENT = "session_schema_agent_xyz"
MODEL_NAME = "gemini-2.0-flash"

## <b><font color='darkblue'>Single-agent system</font></b>
<font size='3ptx'><b>A single-agent system uses an AI model, a defined set of tools, and a comprehensive system prompt to autonomously handle a user request or to complete a specific task.</b> In this fundamental pattern, the agent relies on the model's reasoning capabilities to interpret a user's request, plan a sequence of steps, and decide which tools to use from a defined set.</font>

<b>The system prompt shapes the agent's behavior by defining its core task, persona, and operations, and the specific conditions for using each tool</b>. The following diagram shows a high-level view of a single agent pattern:

![single agent workflow](https://cloud.google.com/static/architecture/images/choose-design-pattern-agentic-ai-system-single-agent.svg)

<font size='3ptx'><b>A single-agent system is ideal for tasks that require multiple steps and access to external data.</b></font> For example, a customer support agent must query a database to find an order status, or a research assistant needs to call APIs to summarize recent news. A non-agentic system can't perform these tasks because it can't autonomously use tools or execute a multi-step plan to synthesize a final answer.

<font size='3ptx'><b>If you're early in your agent development, we recommend that you start with a single agent</b></font>. When you start your agent development with a single-agent system, you can focus on refining the core logic, prompt, and tool definitions of your agent before adding more complex architectural components.

<font size='3ptx' color='orange'><b>A single agent's performance can be less effective when it uses more tools and when tasks increase in complexity</b></font>. You might observe this as increased latency, incorrect tool selection or use, or a failure to complete the task. You can often mitigate these issues by refining the agent's reasoning process with techniques like the [**Reason and Act (ReAct) pattern**](https://cloud.google.com/architecture/choose-design-pattern-agentic-ai-system#react-pattern).

However, <b>if your workflow requires an agent to manage several distinct responsibilities, these techniques might not be sufficient</b>. For these cases, consider a [**multi-agent system**](https://cloud.google.com/architecture/choose-design-pattern-agentic-ai-system#multi-agent-systems), which can improve resilience and performance by delegating specific skills to specialized agents.

In [1]:
import asyncio
import warnings
warnings.filterwarnings('ignore')

### <b><font color='darkgreen'>Example - Count words and classify sentimental result</font></b>
We implement a single agent to count the word and analyze the sentiment (`Positive`, `Negative` and `Neutral`) of the given text:

In [2]:
import single_agent_ex1

Invalid config for agent sentimental_analysis_agent: output_schema cannot co-exist with agent transfer configurations. Setting disallow_transfer_to_parent=True, disallow_transfer_to_peers=True


In [3]:
runner, session_service = await single_agent_ex1.create_runners()

session_service: <class 'google.adk.sessions.in_memory_session_service.InMemorySessionService'>


In [4]:
result = await single_agent_ex1.call_agent(
      "This movie is absolute awesome!",
      runner,
      session_service)

In [5]:
result

{'word_count': 5,
 'result': 'positive',
 'reason': "The text expresses excitement and positive sentiment with the word 'awesome'."}

In [6]:
await single_agent_ex1.call_agent(
      "This car does not deserve nothing and totally a waste of my money.",
      runner,
      session_service)

{'word_count': 13,
 'result': 'negative',
 'reason': "The text expresses negative sentiment due to phrases like 'does not deserve nothing' and 'waste of my money.'"}

In [6]:
await single_agent_ex1.call_agent(
      "This book is normal and nothing special to say about it.",
      runner,
      session_service)

{'word_count': 11,
 'result': 'Neutral',
 'reason': "The text expresses a neutral sentiment. The phrase 'nothing special' indicates a lack of strong positive or negative feelings."}

## <b><font color='darkblue'>Multi-agent systems</font></b>
<font size='3ptx'><b>A multi-agent system orchestrates multiple specialized agents to solve a complex problem that a single agent can't easily manage</b>. The core principle is to decompose a large objective into smaller sub-tasks and assign each sub-task to a dedicated agent with a specific skill. These agents then interact through collaborative or hierarchical workflows to achieve the final goal. <b>Multi-agent patterns provide a modular design that can improve the scalability, reliability, and maintainability of the overall system compared to a single agent with a monolithic prompt.</b></font>

<b><font size='3ptx'>In a multi-agent system, each agent requires a specific context to perform its task effectively</font></b>. Context can include documentation, historical preferences, relevant links, conversational history, or any operational constraints. <b>The process of managing this information flow is called context engineering</b>. [**Context engineering**](https://www.promptingguide.ai/guides/context-engineering-guide#what-is-context-engineering) includes strategies such as isolating context for a specific agent, persisting information across multiple steps, or compressing large amounts of data to improve efficiency.

<b><font size='3ptx'>Building a multi-agent system requires additional evaluation, security, reliability, and cost considerations when compared to a single-agent system</font></b>. For example, multi-agent systems must implement precise access controls for each specialized agent, design a robust orchestration system to ensure reliable inter-agent communication, and manage the increased operational costs from the computational overhead of running multiple agents. For an example reference architecture to build a multi-agent system, see [**Multi-agent AI systems in Google Cloud**](https://cloud.google.com/architecture/multiagent-ai-system).

### <b><font color='darkgreen'>Sequential pattern</font></b>
<font size='3ptx'><b>The multi-agent sequential pattern executes a series of specialized agents in a predefined, linear order where the output from one agent serves as the direct input for the next agent</b>. This pattern uses a [**sequential workflow agent**](https://google.github.io/adk-docs/agents/workflow-agents/sequential-agents/) that operates on predefined logic without having to consult an AI model for the orchestration of its subagents.</font>

The following diagram shows a high-level view of a multi-agent sequential pattern:
![multi-agent sequential pattern](https://cloud.google.com/static/architecture/images/choose-design-pattern-agentic-ai-system-sequential.svg)

<b><font size='3ptx'>Use the sequential pattern for highly structured, repeatable processes where the sequence of operations doesn't change</font></b>. For example, a data processing pipeline might use this pattern to first have a data extraction agent pull raw data, then pass that data to a data cleaning agent for formatting, which in turn passes the clean data to a data loading agent to save it in a database.

<font size='3ptx'><b>The sequential pattern can reduce latency and operational costs compared to a pattern that uses an AI model to orchestrate task workflow</b>. However, <font color='orange'>this efficiency comes at the cost of flexibility</font></font>. The rigid, predefined structure of the pipeline makes it difficult to adapt to dynamic conditions or to skip unnecessary steps, which can cause inefficient processing or lead to higher cumulative latency if an unneeded step is slow.

#### <b>Example - Code Development Pipeline</b>
([source](https://google.github.io/adk-docs/agents/workflow-agents/sequential-agents/#full-example-code-development-pipeline)) Here we will implement a simplified code development pipeline which:
- **Code Writer Agent**: An LLM Agent that generates initial code based on a specification.
- **Code Reviewer Agent**: An LLM Agent that reviews the generated code for errors, style issues, and adherence to best practices. It receives the output of the Code Writer Agent.
- **Code Refactorer Agent**: An LLM Agent that takes the reviewed code (and the reviewer's comments) and refactors it to improve quality and address issues.

### <b><font color='darkgreen'>Parallel pattern</font></b>
<font size='3ptx'><b>The multi-agent parallel pattern, also known as a concurrent pattern, multiple specialized subagents perform a task or sub-tasks independently at the same time.</b> The outputs of the subagents are then synthesized to produce the final consolidated response.</font>

Similar to a [**sequential pattern**](https://cloud.google.com/architecture/choose-design-pattern-agentic-ai-system#sequential-pattern), the parallel pattern uses a [**parallel workflow agent**](https://google.github.io/adk-docs/agents/workflow-agents/parallel-agents/) to manage how and when the other agents run without having to consult an AI model to orchestrate its subagents.

The following diagram shows a high-level view of a multi-agent parallel pattern:

![multi-agent parallel pattern](https://cloud.google.com/static/architecture/images/choose-design-pattern-agentic-ai-system-parallel.svg)

<b><font size='3ptx'>Use the parallel pattern when sub-tasks can be executed concurrently to reduce latency or gather diverse perspectives, such as gathering data from disparate sources or evaluating several options at once</font></b>. For example, to analyze customer feedback, a parallel agent might fan out a single feedback entry to four specialized agents at the same time: a sentiment analysis agent, a keyword extraction agent, a categorization agent, and an urgency detection agent. A final agent gathers these four outputs into a single, comprehensive analysis of that feedback.

<b><font size='3ptx'>The parallel pattern can reduce overall latency compared to a sequential approach because it can gather diverse information from multiple sources at the same time.</font></b> However, this approach introduces trade-offs in cost and complexity. Running multiple agents in parallel can increase immediate resource utilization and token consumption, which leads to higher operational costs. Furthermore, the gather step requires complex logic to synthesize potentially conflicting results, which adds to the development and maintenance overhead of the system.

### <b><font color='darkgreen'>Loop pattern</font></b>
<font size='3ptx'><b>The multi-agent loop agent pattern repeatedly executes a sequence of specialized subagents until a specific termination condition is met.</b> This pattern uses a [**loop workflow agent**](https://google.github.io/adk-docs/agents/workflow-agents/loop-agents) that, like other workflow agents, operates on predefined logic without consulting an AI model for orchestration. After all of the subagents complete their tasks, the loop agent evaluates whether an exit condition is met.</font> 

<b><font size='3ptx'>The condition can be a maximum number of iterations or a custom state</font></b>. If the exit condition isn't met, then the loop agent starts the sequence of subagents again. You can implement a loop pattern where the exit condition is evaluated at any point in the flow. Use the loop pattern for tasks that require iterative refinement or self-correction, such as generating content and having a critic agent review it until it meets a quality standard.

The following diagram shows a high-level view of a multi-agent loop pattern:

![multi-agent loop pattern](https://cloud.google.com/static/architecture/images/choose-design-pattern-agentic-ai-system-loop.svg)

The loop agent pattern provides a way to build complex, iterative workflows. It enables agents to refine their own work and continue processing until a specific quality or state is achieved. However, <b><font size='3ptx' color='orange'>this pattern's primary trade-off is the risk of an infinite loop. If the termination condition isn't correctly defined or if the subagents fail to produce the state that's required to stop, the loop can run indefinitely.</font></b> This can lead to excessive operational costs, high resource consumption, and potential system hangs.

### <b><font color='darkgreen'>Review and critique pattern</font></b>
<font size='3ptx'><b>The multi-agent review and critique pattern, also known as the <font color='darkblue'>generator and critic pattern</font>, improves the quality and reliability of generated content by using two specialized agents, typically in a sequential workflow</b>. The review and critique pattern is an implementation of the [**loop agent pattern**](https://cloud.google.com/architecture/choose-design-pattern-agentic-ai-system#loop-pattern).</font>

<font size='3ptx'><b>In the review and critique pattern, a generator agent creates an initial output, such as a block of code or a summary of a document</b></font>. Next, a critic agent evaluates this output against a predefined set of criteria, such as factual accuracy, adherence to formatting rules, or safety guidelines. Based on the evaluation, the critic can approve the content, reject it, or return it to the generator with feedback for revision.

The following diagram shows a high-level view of a multi-agent review and critique pattern:

![multi-agent review and critique pattern](https://cloud.google.com/static/architecture/images/choose-design-pattern-agentic-ai-system-review-critique.svg)

<b><font size='3ptx'>This pattern is suitable for tasks where outputs must be highly accurate or must conform to strict constraints before they're presented to a user or used in a downstream process</font></b>. For example, in a code generation workflow, a generator agent might write a function to fulfill a user's request. This generated code is then passed to a critic agent that acts as a security auditor. The critic agent's job is to check the code against a set of constraints, such as scanning for security vulnerabilities or verifying that it passes all of the unit tests, before the code is approved for use.

The reviewer and critique pattern can improve output quality, accuracy, and reliability because it adds a dedicated verification step. However, <b><font size='3ptx' color='orange'>this quality assurance comes at the direct cost of increased latency and operational expenses</font></b>. The workflow requires at least one additional model call for the critic's evaluation. If the process includes revision loops where content is sent back for refinement, then both latency and costs accumulate with each iteration.

### <b><font color='darkgreen'>Iterative refinement pattern</font></b>
<font size='3ptx'><b>The iterative refinement pattern uses a looping mechanism to progressively improve an output over multiple cycles.</b> The iterative refinement pattern is an implementation of the [**loop agent pattern**](https://cloud.google.com/architecture/choose-design-pattern-agentic-ai-system#loop-pattern).</font>

<b><font size='3ptx'>In this pattern, one or more agents work within a loop to modify a result that's stored in the session state during each iteration</font></b>. The process continues until the output meets a predefined quality threshold or it reaches a maximum number of iterations, which prevents infinite loops.

The following diagram shows a high-level view of a multi-agent iterative refinement pattern:

![multi-agent iterative refinement pattern](https://cloud.google.com/static/architecture/images/choose-design-pattern-agentic-ai-system-iterative-refinement.svg)

This pattern is suitable for complex generation tasks where the output is difficult to achieve in a single step. Examples of such tasks include writing and debugging a piece of code, developing a detailed multi-part plan, or drafting and revising a long-form document. For example, in a creative writing workflow, an agent might generate a draft of a blog post, critique the draft for flow and tone, and then rewrite the draft based on that critique. This process repeats in a loop until the agent's work meets a predefined quality standard or until the repetition reaches a maximum number of iterations.

The iterative refinement pattern can produce highly complex or polished outputs that would be difficult to achieve in a single step. However, <b><font size='3ptx' color='orange'>the looping mechanism directly increases latency and operational costs with each cycle</font></b>. This pattern also adds architectural complexity, because it requires carefully designed exit conditions—such as a quality evaluation or a maximum iteration limit—to prevent excessive costs or uncontrolled execution.

### <b><font color='darkgreen'>Coordinator pattern</font></b>
<font size='3ptx'><b>The multi-agent coordinator pattern uses a central agent, the coordinator, to direct a workflow</b>. The coordinator analyzes and decomposes a user's request into sub-tasks, and then it dispatches each sub-task to a specialized agent for execution. Each specialized agent is an expert in a specific function, such as querying a database or calling an API</font>.

<b><font size='3ptx'>A distinction of the coordinator pattern is its use of an AI model to orchestrate and dynamically route tasks</font></b>. By contrast, the [**parallel pattern**](https://cloud.google.com/architecture/choose-design-pattern-agentic-ai-system#parallel-pattern) relies on a hardcoded workflow to dispatch tasks for simultaneous execution without the need for AI model orchestration.

The following diagram shows a high-level view of a multi-agent coordinator pattern:

![multi-agent coordinator pattern](https://cloud.google.com/static/architecture/images/choose-design-pattern-agentic-ai-system-coordinator.svg)

<b><font size='3ptx'>Use the coordinator pattern for automating structured business processes that require adaptive routing</font></b>. For example, a customer service agent can act as the coordinator. The coordinator agent analyzes the request to determine whether it's an order status request, product return, or refund request. Based on the type of request, the coordinator routes the task to the appropriate specialized agent.

The coordinator pattern offers flexibility compared to more rigid, predefined workflows. By using a model to route tasks, the coordinator can handle a wider variety of inputs and adapt the workflow at runtime. However, <b><font size='3ptx' color='orange'>this approach also introduces trade-offs. Because the coordinator and each specialized agent rely on a model for reasoning, this pattern results in more model calls than a single-agent system</font></b>. Although the coordinator pattern can lead to higher-quality reasoning, it also increases token throughput, operational costs, and overall latency when compared to a single-agent system.

### <b><font color='darkgreen'>Hierarchical task decomposition pattern</font></b>
<font size='3ptx'><b>The multi-agent hierarchical task decomposition pattern organizes agents into a multi-level hierarchy to solve complex problems that require extensive planning</b>. The hierarchical task decomposition pattern is an implementation of the coordinator pattern. A top-level parent, or root, agent receives a complex task and it's responsible for decomposing the task into several smaller, manageable sub-tasks.</font> 

<b><font size='3ptx'>The root agent delegates each sub-task to a specialized subagent at a lower level</font></b>. This process can repeat through multiple layers, with agents that progressively decompose their assigned tasks until the tasks are simple enough for a worker agent at the lowest level to execute directly.

The following diagram shows a high-level view of a multi-agent hierarchical task decomposition pattern:

![multi-agent hierarchical task decomposition pattern](https://cloud.google.com/static/architecture/images/choose-design-pattern-agentic-ai-system-hierarchical-task.svg)

<b><font size='3ptx'>Use the hierarchical task decomposition pattern for ambiguous, open-ended problems that require multi-step reasoning, such as tasks that involve research, planning, and synthesis</font></b>. For example, to complete a complex research project, a coordinator agent decomposes the high-level goal into multiple tasks such as gathering information, analyzing the findings, and synthesizing the final report. The coordinator agent then delegate those tasks to specialized subagents, such as an agent for data gathering, an analysis agent, and an agent that writes reports, to execute or further decompose.

The hierarchical task decomposition pattern is ideal for solving highly complex and ambiguous problems because <b><font size='3ptx'>it systematically decomposes them into manageable sub-tasks. This pattern can result in more comprehensive and higher-quality results than simpler patterns</font></b>. However, <b><font size='3ptx' color='orange'>this advanced capability introduces significant trade-offs. The multi-level structure adds considerable architectural complexity, which makes the system more difficult to design, debug, and maintain. The multiple layers of delegation and reasoning also result in a high number of model calls, which significantly increases both overall latency and operational costs</font></b> compared to other patterns.

### <b><font color='darkgreen'>Swarm pattern</font></b>
<font size='3ptx'><b>The multi-agent swarm pattern uses a collaborative, all-to-all communication approach</b>. In this pattern, multiple specialized agents work together to iteratively refine a solution to a complex problem.</font>

The following diagram shows a high-level view of a multi-agent swarm pattern:

![multi-agent swarm pattern](https://cloud.google.com/static/architecture/images/choose-design-pattern-agentic-ai-system-swarm.svg)

<b><font size='3ptx'>The swarm pattern uses a dispatcher agent to route a user request to a collaborative group of specialized agents</font></b>. The dispatcher agent interprets the request and it determines which agent in the swarm is best suited to begin the task. <b>In this pattern, each agent can communicate with every other agent, which allows them to share findings, critique proposals, and build upon each other's work to iteratively refine a solution</b>. Any agent in the swarm can hand off the task to another agent that it determines is better suited to handle the next step, or it can communicate the final response back to the user through the coordinator agent.

<b><font size='3ptx'>A swarm typically lacks a central supervisor or coordinator agent to keep the process on track</font></b>. The dispatcher agent doesn't orchestrate the agentic workflow, unlike the coordinator pattern. Instead, the dispatcher agent facilitates communication between the swarm subagents and the user. <b>To ensure that the swarm eventually stops and returns a result, you must define an explicit exit condition. This condition is often a maximum number of iterations, a time limit, or the achievement of a specific goal, such as reaching a consensus</b>.

<b><font size='3ptx'>Use the swarm pattern for ambiguous or highly complex problems that benefit from debate and iterative refinement</font></b>. For example, designing a new product could involve a market researcher agent, an engineering agent, and a financial modeling agent. The agents would share initial ideas, debate the trade-offs between features and costs, and collectively converge on a final design specification that balances all of the competing requirements.

<b><font size='3ptx'>The swarm pattern simulates a collaborative team of experts, therefore it can produce exceptionally high-quality and creative solutions</font></b>. However, it represents the most complex and costly multi-agent pattern to implement. <b><font color='orange'>The lack of an agent that uses an AI model to orchestrate can introduce the risk of unproductive loops or the failure to converge on a solution.</font></b> You must therefore design sophisticated logic to manage the intricate inter-agent communication, control the iterative workflow, and handle the significant operational costs and latency that are associated with running a dynamic, multi-turn conversation between multiple agents.

## <b><font color='darkblue'>Reason and act (ReAct) pattern</font></b>
<font size='3ptx'><b>The ReAct pattern is an approach that uses the AI model to frame its thought processes and actions as a sequence of natural language interactions</b>.</font> In this pattern, the agent operates in an iterative loop of thought, action, and observation until an exit condition is met.
* <b><font size='3ptx'>Thought</font></b>: The model reasons about the task and it decides what to do next. The model evaluates all of the information that it's gathered in order to determine whether the user's request has been fully answered.
* <b><font size='3ptx'>Action</font></b>: Based on its thought process, the model takes one of two actions:
    - If the task isn't complete, it selects a tool and then it forms a query to gather more information.
    - If the task is complete, it formulates the final answer to send to the user, which ends the loop.
* <b><font size='3ptx'>Observation</font></b>: The model receives the output from the tool and it saves relevant information in its memory. Because the model saves relevant output, it can build on previous observations, which helps to prevent the model from repeating itself or losing context.

<b><font size='3ptx'>The iterative loop terminates when the agent finds a conclusive answer, reaches a preset maximum number of iterations, or encounters an error that prevents it from continuing</font></b>. This iterative loop lets the agent dynamically build a plan, gather evidence, and adjust its approach as it works toward a final answer.

The following diagram shows a high-level view of the ReAct pattern:

![multi-agent ReAct pattern](https://cloud.google.com/static/architecture/images/choose-design-pattern-agentic-ai-system-react.svg)

Use the ReAct pattern for complex, dynamic tasks that require continuous planning and adaptation. For example, consider a robotics agent that must generate a path to transition from an initial state to a goal state:
* <b><font size='3ptx'>Thought</font></b>: The model reasons about the optimal path to transition from its current state to the goal state. During the thought process, the model optimizes for metrics like time or energy.
* <b><font size='3ptx'>Action</font></b>: The model executes the next step in its plan by moving along a calculated path segment.
* <b><font size='3ptx'>Observation</font></b>: The model observes and saves the new state of the environment. The model saves its new position and any changes to the environment that it perceives.

<b><font size='3ptx'>This loop allows the agent to adhere to dynamic constraints, such as avoiding new obstacles or following traffic regulations, by constantly updating its plan based on new observations</font></b>. The agent continues through its iterative loop until it reaches its goal or encounters an error.

<b><font size='3ptx'>A single ReAct agent can be simpler and more cost-effective to implement and maintain than a complex multi-agent system</font></b>. Model thinking provides a transcript of the model's reasoning, which helps with debugging. However, <font color='orange'><b>this flexibility introduces trade-offs. The iterative, multi-step nature of the loop can lead to higher end-to-end latency compared to a single query</b></font>. Furthermore, the agent's effectiveness is highly dependent on the quality of the AI model's reasoning. Therefore, <font color='orange'><b>an error or a misleading result from a tool in one observation step can propagate and cause the final answer to be incorrect</b></font>.

## <b><font color='darkblue'>Human-in-the-loop pattern</font></b>
<font size='3ptx'><b>The human-in-the-loop pattern integrates points for human intervention directly into an agent's workflow</b>. At a predefined checkpoint, the agent pauses its execution and calls an external system to wait for a person to review its work. This pattern lets a person approve a decision, correct an error, or provide necessary input before the agent can continue.</font>

The following diagram shows a high-level view of a human-in-the-loop pattern:

![multi-agent human-in-the-loop pattern](https://cloud.google.com/static/architecture/images/choose-design-pattern-agentic-ai-system-human-in-the-loop.svg)

<b><font size='3ptx'>Use the human-in-the-loop pattern for tasks that require human oversight, subjective judgment, or final approval for critical actions</font></b>. Such actions include approving a large financial transaction, validating the summary of a sensitive document, or providing subjective feedback on generated creative content. For example, an agent might be tasked with anonymizing a patient dataset for research. The agent would automatically identify and redact all protected health information, but it would pause at a final checkpoint. It would then wait for a human compliance officer to manually validate the dataset and approve its release, which helps to ensure that no sensitive data is exposed.

The human-in-the-loop pattern improves safety and reliability by inserting human judgment into critical decision points within the workflow. <b><font color='orange' size='3ptx'>This pattern can add significant architectural complexity because it requires you to build and maintain the external system for user interaction.</font></b>

## <b><font color='darkblue'>Custom logic pattern</font></b>
<font size='3ptx'><b>The custom logic pattern provides the maximum flexibility in your workflow design.</b> This approach lets you implement specific orchestration logic that uses code, such as conditional statements, to create complex workflows with multiple branching paths.</font>

The following diagram illustrates an example use of a custom logic pattern to capture a refund process:

![multi-agent custom logic pattern](https://cloud.google.com/static/architecture/images/choose-design-pattern-agentic-ai-system-custom.svg)

In the preceding diagram, the following is the agentic workflow for the example customer refund agent:
1. The user <b>sends a query to the customer refund agent that acts as a coordinator agent</b>.
2. The coordinator's custom logic first invokes a parallel verifier agent, which <b>simultaneously dispatches two subagents: the purchaser verifier agent and the refund eligibility agent</b>.
3. After the results are gathered, the coordinator agent executes a tool to <b>check whether the request is eligible for a refund</b>.
     - If the user is eligible, then the coordinator routes the task to a refund processor agent, which calls the process_refund tool.
     - If the user isn't eligible, then the coordinator routes the task to a separate sequential flow, starting with the store credit agent and the process credit decision agent.
4. The result from whichever path is taken is <b>sent to the final response agent to formulate the answer for the user</b>.

<b><font size='3ptx'>The customer refund agent example requires a unique solution for its logic-level orchestration, which goes beyond the structured approaches that other patterns offer</font></b>. This workflow mixes patterns because it runs a parallel check, and then it executes a custom conditional branch that routes to two entirely different downstream processes. <b>This type of complex, mixed-pattern workflow is the ideal use case for the custom logic pattern</b>.

<b><font size='3ptx'>Use the custom logic pattern when you need fine-grained control over the agent's execution or when your workflow doesn't fit one of the other patterns that's described in this document.</font></b> However, this approach <b><font color='orange'>increases development and maintenance complexity</font></b>. You are responsible for designing, implementing, and debugging the entire orchestration flow, which requires more development effort and can be more error-prone than using a predefined pattern that is supported by an agent development tool like [**Agent Development Kit (ADK)**](https://google.github.io/adk-docs/).

For information about custom agents and how to implement custom logic using ADK, see [**Custom agents**](https://google.github.io/adk-docs/agents/custom-agents/).

## <b><font color='darkblue'>Compare design patterns</font></b>
<font size='3ptx'><b>Choosing an agent pattern is a fundamental architectural decision</b>. Each pattern offers different trade-offs in flexibility, complexity, and performance. To determine the appropriate pattern for your workload, consider the design patterns in the following sections.</font>

### <b><font color='darkgreen'>Workflows that are deterministic</font></b>
<font size='3ptx'><b>Workflow that are deterministic include tasks that are predictable and sequential, and that have a clearly defined workflow path from start to finish.</b> The steps in your tasks are known in advance, and the process doesn't change much from one run to the next. </font>

The following are agent design patterns for workflows that are deterministic:

| <font size='3ptx'>Workload characteristics</font> | <font size='3ptx'>Agent design pattern</font> |
| :--- | :--- |
| * Multi-step tasks that follow a predefined, **rigid workflow**. | [**Multi-agent sequential pattern**](#Sequential-pattern) |
| * Doesn't require **model orchestration**. | |
| * **Fixed sequence of operations**. The output of one agent is the direct input of the next agent in the sequence. | |
| * **Independent tasks** that can be executed at the same time. | [**Multi-agent parallel pattern**](#Parallel-pattern) |
| * Doesn't require model orchestration. | |
| * **Reduces overall latency** by running sub-tasks simultaneously. | |
| * **Open-ended or complex generation tasks** that are difficult to complete in a single attempt. | [**Multi-agent iterative refinement pattern**](#Iterative-refinement-pattern) |
| * **Requires the agent to progressively improve the output over multiple cycles**. | |
| * Doesn't require model orchestration. | |
| * **Prioritizes output quality over latency**. | |

### <b><font color='darkgreen'>Workflows that require dynamic orchestration</font></b>
<font size='3ptx'><b>Workflows that require dynamic orchestration include complex problems where the agents must determine the best way to proceed</b>. The agentic AI system needs to dynamically plan, delegate, and coordinate tasks without a predefined script.</font>

The following are agent design patterns for workflows that require autonomous and dynamic orchestration:

| Workload characteristics | Agent design pattern |
| :--- | :--- |
| * **Structured and multi-step tasks** that require the use of **external tools**. | [**Single agent pattern**](#Single-agent-system) |
| * Requires **fast development** for a prototype of a solution as a proof of concept. | |
| * **Requires dynamic routing** to an appropriate specialized subagent for structured tasks with varied input. | [**Multi-agent coordinator pattern**](#Coordinator-pattern) |
| * **High latency** due to multiple calls to the coordinator AI model so that it can direct tasks to the appropriate subagent. | |
| * Can incur **high cost** due to multiple calls to the coordinator agent. | |
| * **Requires multi-level model orchestration** for complex, open-ended, and ambiguous tasks. | [**Multi-agent hierarchical task decomposition pattern**](#Hierarchical-task-decomposition-pattern) |
| * Requires **comprehensive, high-quality results** where **decomposing ambiguity is the primary challenge**. | |
| * **High latency** due to nested, multi-level decomposition that leads to multiple calls to the AI model for reasoning. | |
| * Requires **collaborative debate and iterative refinement** from multiple specialized agents for highly complex, open-ended, or ambiguous tasks. | [**Multi-agent swarm pattern**](#Swarm-pattern) |
| * Prioritizes the **synthesis of multiple perspectives** to create a comprehensive or creative solution. | |
| * **High latency and operational costs** due to dynamic, all-to-all communication between agents. | |

### <b><font color='darkgreen'>Workflows that involve iteration</font></b>
<font size='3ptx'>Workflows that involve iteration include tasks where the final output is achieved through cycles of refinement, feedback, and improvement. </font>

The following are agent design patterns for workflows that involve iteration:

| Workload characteristics | Agent design pattern |
| :--- | :--- |
| * Requires an agent to **iteratively reason, act, and observe** to build or adapt a plan for complex, open-ended, and dynamic tasks. | [**ReAct pattern**](#Reason-and-act-(ReAct)-pattern) |
| * Prioritizes a more **accurate and thorough result over latency**. | |
| * Requires **monitoring or polling tasks** that repeat a predefined action, such as automated checks, until the agent meets an **exit condition**. | [**Multi-agent loop pattern**](#Loop-pattern) |
| * **Unpredictable or long-running latency** while waiting for an exit condition to be met. | |
| * Tasks require a **distinct validation step before completion**. | [**Multi-agent review and critique pattern**](#Review-and-critique-pattern) |
| * **Open-ended or complex generation tasks** that are difficult to complete in a single attempt. | [**Multi-agent iterative refinement pattern**](#Iterative-refinement-pattern) |
| * Requires the agent to **progressively improve the output over multiple cycles**. | |
| * Doesn't require model orchestration. | |
| * Prioritizes **output quality over latency**. | |

### <b><font color='darkgreen'>Workflows that have special requirements</font></b>
<font size='3ptx'>Workflows that have special requirements include tasks that don't follow the common agentic patterns. <b>Your tasks can include unique business logic or they can require human judgment and intervention at critical points</b>. Your agentic AI system is a custom-built machine designed for a single, specific purpose.</font>

The following are agent design patterns for workflows that have special requirements:

| Workload characteristics | Agent design pattern |
| :--- | :--- |
| * Requires **human supervision** due to high-stakes or subjective tasks that might include **safety, reliability, and compliance** requirements. | [**Human-in-the-loop pattern**](#Human-in-the-loop-pattern) |
| * **Complex, branching logic** that goes beyond a direct linear sequence. | [**Custom logic pattern**](#Custom-logic-pattern) |
| * Requires maximum control to **mix predefined rules with model reasoning**. | |
| * Requires **fine-grained process control** for a workflow that doesn't fit a standard template. | |

## <b><font color='darkblue'>What's next</font></b>
* Learn more about how to construct and [**manage multi-agent systems using ADK primitives**](https://google.github.io/adk-docs/agents/multi-agents/).
* Learn how to [**host AI apps and agents on Cloud Run**](https://cloud.google.com/run/docs/ai-agents).
* Learn more about Agentic Design Patterns: [**A Hands-On Guide to Building Intelligent Systems**](https://www.amazon.com/Agentic-Design-Patterns-Hands-Intelligent/dp/3032014018/).
* Learn how to [**build an agent with the Agent Development Kit**](https://cloud.google.com/vertex-ai/generative-ai/docs/agent-development-kit/quickstart).
* Learn more about how to [**build multi-agent AI systems in Google Cloud**](https://cloud.google.com/architecture/multiagent-ai-system).
* For more reference architectures, diagrams, and best practices, [**explore the Cloud Architecture Center**](https://cloud.google.com/architecture).